This code captures video from the camera, detects faces in the frames, and adjusts the servo motor's angle to track the detected face. It stops when you press 'q' in the OpenCV window.

In [10]:

!python -m  pip install RPi.GPIO

  Using cached RPi.GPIO-0.7.1.tar.gz (29 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for RPi.GPIO
Failed to build RPi.GPIO


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [12 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build\lib.win-amd64-cpython-310
      creating build\lib.win-amd64-cpython-310\RPi
      copying RPi\__init__.py -> build\lib.win-amd64-cpython-310\RPi
      creating build\lib.win-amd64-cpython-310\RPi\GPIO
      copying RPi\GPIO\__init__.py -> build\lib.win-amd64-cpython-310\RPi\GPIO
      running build_ext
      building 'RPi._GPIO' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for RPi.GPIO
ERROR: Could not build wheels for RPi.GPIO, which is required to install pyproject.to

In [11]:
import cv2
import RPi.GPIO as GPIO  # For Raspberry Pi

# Initialize GPIO (for Raspberry Pi)
GPIO.setmode(GPIO.BOARD)
servo_pin = 18  # GPIO pin for servo motor control
GPIO.setup(servo_pin, GPIO.OUT)
servo = GPIO.PWM(servo_pin, 50)  # PWM frequency: 50Hz
servo.start(0)  # Start with duty cycle of 0

# Initialize the face detection cascade
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Initialize the camera
cap = cv2.VideoCapture(0)

# Set initial servo angle
current_angle = 90  # 90 degrees is the center position
servo.ChangeDutyCycle(current_angle / 18 + 2)

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Convert the frame to grayscale for face detection
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detect faces in the frame
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        for (x, y, w, h) in faces:
            # Calculate the center of the detected face
            center_x = x + w // 2
            center_y = y + h // 2

            # Calculate the angle to move the servo
            angle = ((center_x / frame.shape[1]) - 0.5) * 90

            # Update the servo position
            servo.ChangeDutyCycle(angle / 18 + 2)

            # Draw a rectangle around the detected face
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Display the frame
        cv2.imshow('Face Recognition', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:
    # Cleanup
    servo.stop()
    GPIO.cleanup()  # Release GPIO resources
    cap.release()
    cv2.destroyAllWindows()

ModuleNotFoundError: No module named 'RPi'